# Workspace for doing exploration testing and analysis

## Imports and some helper functions

In [17]:
from tango import DeviceProxy, DevState, CmdArgType
from numpy import ndarray
class resource:

    device_name = None

    def __init__(self,device_name):
        self.device_name = device_name
    
    def get(self,attr):
        p = DeviceProxy(self.device_name)
        attrs = p.get_attribute_list()
        if (attr not in attrs) : return "attribute not found"
        tp = p._get_attribute_config(attr).data_type
        if (tp == CmdArgType.DevEnum):
            return getattr(p,attr).name
        if (tp == CmdArgType.DevState):
            return str(p.read_attribute(attr).value)
        else:
            value = getattr(p,attr)
            if isinstance(value,ndarray):
                return tuple(value)
            return getattr(p,attr)

In [3]:
def update_file(file):
    import json
    import random
    from datetime import date 
    with open(file,'r') as f: 
        data = json.load(f)
    data['sdp']['configure'][0]['id'] = "realtime-"+date.today().strftime("%Y%m%d")+"-"+str(random.choice(range(1, 10000))) 
    with open(file,'w') as f:
        json.dump(data, f)
file = "sp354/polaris_b1_no_cam.json"

In [4]:
def show(par):
    out.clear_output()
    with out:
        for n in par['names']:
            l = [n]
            for a in par['att']:
                l.append(a)
                l.append(resource(n).get(a))
            t = tuple(l)
            print('%s is in %s %s and %s %s'%t)

In [5]:
def read(dev,att):
    with out:
        val=resource(dev).get(att)
        print('%s %s is %s'%(dev,att,val))

In [6]:
par1 = {'names':['ska_mid/tm_subarray_node/1',
'mid_csp/elt/subarray_01',
'mid_sdp/elt/subarray_1',
'mid_csp_cbf/sub_elt/subarray_01'],
'att':['obsState'
,'State']}

### Analyse latencies

*Create an array of dishmasters - replicated according to the total nr required for the test*

In [19]:
dishnames = ['mid_d0001/elt/master','mid_d0002/elt/master','mid_d0003/elt/master','mid_d0004/elt/master']
replicated_dishes = [dishnames[x % 4] for x in range(1000)]

In [26]:
total_time = 0
for x in range(1000):
    dish_proxy = DeviceProxy(replicated_dishes[x])
    total_time += dish_proxy.ping()

In [28]:
print((total_time)/(1000*1000))

0.231655


In [39]:
import time
total_time = 0
for x in range(1000):
    dish_proxy = DeviceProxy(replicated_dishes[x])
    start = time.time()
    dummy_value = dish_proxy.achievedPointing
    end = time.time()
    total_time += (end - start)

In [40]:
print(total_time)

3.5203804969787598


## Tests observations

*display console for checking the states*

In [7]:
import ipywidgets as widgets
out = widgets.Output(layout={'border': '1px solid black'})
out

Output(layout=Layout(border='1px solid black'))

*creation of objects for controlling system*

In [ ]:
telescope = SKAMid()
subarray = SubArray(1)

In [ ]:
show(par1)

In [ ]:
show(par1)
read('ska_mid/tm_subarray_node/1','receptorIDList')
read('mid_csp/elt/subarray_01','receptors')
read('mid_csp/elt/master','receptorMembership')

### Run a Test

In [ ]:
telescope.start_up()
show(par1)

In [ ]:
subarray.allocate(ResourceAllocation(dishes=[Dish(1), Dish(2), Dish(3), Dish(4)]))
show(par1)
read('ska_mid/tm_subarray_node/1','receptorIDList')
read('mid_csp/elt/subarray_01','receptors')
read('mid_csp/elt/master','receptorMembership')

In [ ]:
update_file(file)
subarray.configure_from_file(file,false)
show(par1)

In [ ]:
subarray.end_sb()

In [ ]:
subarray.deallocate()
show(par1)
read('ska_mid/tm_subarray_node/1','receptorIDList')
read('mid_csp/elt/subarray_01','receptors')
read('mid_csp/elt/master','receptorMembership')

In [ ]:
telescope.standby()
show(par1)

## Testing the OET test runner (OET cluster)

### Help Information

**General help information can be obtained by typing the command:**

In [1]:
!oet

NAME
    oet - OET script execution client.

SYNOPSIS
    oet - COMMAND

DESCRIPTION
    This client can be used to connect to a remote OET script execution
    service. With this client, you can create procedures, list procedures, and
    start a procedure that is ready for execution.

    Multiple procedures may be initialised, but only one script may be
    running at any one time.

COMMANDS
    COMMAND is one of the following:

     create
       Create a new Procedure.

     list
       List procedures registered on the targeted server.

     start
       Start a specified Procedure.


**Detailed help information for specific commands is also available e.g.:**

In [2]:
!oet create --help

INFO: Showing help with the command 'oet create -- --help'.

NAME
    oet create - Create a new Procedure.

SYNOPSIS
    oet create SCRIPT_URI <flags> [ARGS]...

DESCRIPTION
    Arguments will be passed to the Procedure initialiser.

    Example:

        oet create file://path/to/script.py 'hello' --verbose=true

POSITIONAL ARGUMENTS
    SCRIPT_URI
        script URI, e.g., file://test.py
    ARGS
        script positional arguments

FLAGS
    Flags are accepted.
        script keyword arguments

NOTES
    You can also use flags syntax for POSITIONAL ARGUMENTS


### Examples

This section runs through an example session in which we will load two new ‘Procedures’ and then run one of them. First we load the procedures:

In [3]:
!oet create file:///scripts/hello_world.py 'hello' --verbose=true

  ID  URI                                              Script                          State
----  -----------------------------------------------  ------------------------------  -------
   2  http://rest-oet-test:5000/api/v1.0/procedures/2  file:///scripts/hello_world.py  READY


Note the use of both positional and keyword/value arguments for the procedure on the command line. Now create a second procedure:

In [2]:
!oet create file:///scripts/hello_world.py 'goodbye'

  ID  URI                                              Script                          State
----  -----------------------------------------------  ------------------------------  -------
   1  http://rest-oet-test:5000/api/v1.0/procedures/1  file:///scripts/hello_world.py  READY


We can check the state of the procedures currently loaded by:

In [4]:
!oet list

  ID  URI                                              Script                          State
----  -----------------------------------------------  ------------------------------  -------
   1  http://rest-oet-test:5000/api/v1.0/procedures/1  file:///scripts/hello_world.py  READY
   2  http://rest-oet-test:5000/api/v1.0/procedures/2  file:///scripts/hello_world.py  READY


Alternatively, we could check the state of procedure 2 by typing:

In [5]:
!oet list 2

  ID  URI                                              Script                          State
----  -----------------------------------------------  ------------------------------  -------
   2  http://rest-oet-test:5000/api/v1.0/procedures/2  file:///scripts/hello_world.py  READY


Now that we have our procedures loaded we can start one of them running. At this point we supply the index number of the procedure to run, and some runtime arguments to pass to it if required.

In [8]:
!oet start 2 'bob' --simulate=false

  ID  URI                                              Script                          State
----  -----------------------------------------------  ------------------------------  -------
   1  http://rest-oet-test:5000/api/v1.0/procedures/1  file:///scripts/hello_world.py  RUNNING


A ‘list’ command will give the same information:

In [9]:
!oet list

ID    URI    Script    State
----  -----  --------  -------


### Example session in a SKAMPI environment

*# create process for telescope start-up and execute it*

In [11]:
!oet create file:///scripts/startup.py
!oet start

  ID  URI                                              Script                      State
----  -----------------------------------------------  --------------------------  -------
   1  http://rest-oet-test:5000/api/v1.0/procedures/1  file:///scripts/startup.py  READY
  ID  URI                                              Script                      State
----  -----------------------------------------------  --------------------------  -------
   1  http://rest-oet-test:5000/api/v1.0/procedures/1  file:///scripts/startup.py  RUNNING


*# create process for resource allocation script*

In [12]:
!oet create file:///scripts/allocate.py

  ID  URI                                              Script                       State
----  -----------------------------------------------  ---------------------------  -------
   1  http://rest-oet-test:5000/api/v1.0/procedures/1  file:///scripts/allocate.py  READY


*# run the allocation script, allocating dishes 1-4*

In [14]:
!oet start --dish_ids=1,2,3,4

  ID  URI                                              Script                       State
----  -----------------------------------------------  ---------------------------  -------
   1  http://rest-oet-test:5000/api/v1.0/procedures/1  file:///scripts/allocate.py  RUNNING


*# create process for configure/scan script*

In [15]:
!oet create file:///scripts/observe.py

  ID  URI                                              Script                      State
----  -----------------------------------------------  --------------------------  -------
   1  http://rest-oet-test:5000/api/v1.0/procedures/1  file:///scripts/observe.py  READY


run the script, specifying configuration JSON to applied followed by a 10 second scan. 

Set process_json to False if JSON should be passed through directly to TMC (possibly required for new SDP schema)

In [20]:
!oet start sp354/polaris_b1_no_cam.json 10 --process_json=True

  ID  URI                                              Script                      State
----  -----------------------------------------------  --------------------------  -------
   1  http://rest-oet-test:5000/api/v1.0/procedures/1  file:///scripts/observe.py  RUNNING


create process for resource deallocation script

In [21]:
!oet create file:///scripts/deallocate.py

  ID  URI                                              Script                         State
----  -----------------------------------------------  -----------------------------  -------
   1  http://rest-oet-test:5000/api/v1.0/procedures/1  file:///scripts/deallocate.py  READY


run with no arguments, which requests deallocation of all resources

In [22]:
!oet start

  ID  URI                                              Script                         State
----  -----------------------------------------------  -----------------------------  -------
   1  http://rest-oet-test:5000/api/v1.0/procedures/1  file:///scripts/deallocate.py  RUNNING


create process for telescope standby script

In [24]:
!oet create file:///scripts/standby.py
!oet start

  ID  URI                                              Script                      State
----  -----------------------------------------------  --------------------------  -------
   1  http://rest-oet-test:5000/api/v1.0/procedures/1  file:///scripts/standby.py  READY
  ID  URI                                              Script                      State
----  -----------------------------------------------  --------------------------  -------
   1  http://rest-oet-test:5000/api/v1.0/procedures/1  file:///scripts/standby.py  RUNNING
